In [320]:
### This code cleans Code of Federal Regulations Supplement No. 4 to Part 744, Title 15, from https://www.ecfr.gov/current/title-15/subtitle-B/chapter-VII/subchapter-C/part-744/appendix-Supplement%20No.%204%20to%20Part%20744
### It contains the list of names of certain non-US entities that are subject to specific license requirements for export, re-export or transfer of specified items by bureau of industry and security, US Department of Commerce
### CSV version downloadable from https://www.bis.doc.gov/index.php/documents/consolidated-entity-list?format=html
### For more details, check https://www.bis.doc.gov/index.php/policy-guidance/lists-of-parties-of-concern/entity-list
### A search engine is available on https://www.trade.gov/consolidated-screening-list

import pandas as pd
import numpy as np
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs

In [321]:
## Historical revisions updated 2024-02-21
dates = ['2024-02-16','2024-02-08','2024-01-29','2024-01-25','2024-01-19','2024-01-15','2024-01-10','2023-12-28','2023-12-07','2023-11-21','2023-11-17','2023-11-06','2023-10-19','2023-10-11','2023-09-27','2023-07-19','2023-06-21','2023-06-14','2023-05-22','2023-04-26','2023-04-17','2023-03-30','2023-03-14','2023-03-06','2023-02-27','2023-02-14','2023-02-01','2022-12-23','2022-12-19','2022-12-16','2022-12-08','2022-10-21','2022-10-13','2022-10-07','2022-10-04','2022-09-16','2022-09-09','2022-08-24','2022-06-30','2022-06-06','2022-06-01','2022-05-11','2022-04-11','2022-04-07','2022-03-16','2022-03-09','2022-03-08','2022-03-03','2022-02-14','2022-02-03','2021-12-17','2021-11-26','2021-11-04','2021-10-05','2021-08-20','2021-07-21','2021-07-19','2021-07-12','2021-07-06','2021-06-24','2021-06-16','2021-06-01','2021-04-09','2021-03-16','2021-03-08','2021-03-04','2021-01-15','2020-12-23','2020-12-22','2020-10-30','2020-10-19','2020-09-22','2020-09-11','2020-08-27','2020-08-20','2020-07-22','2020-06-18','2020-06-05','2020-05-19','2020-03-16','2020-03-02','2019-12-18','2019-12-06','2019-11-13','2019-10-21','2019-10-09','2019-08-21','2019-08-14','2019-06-24','2019-05-24','2019-05-21','2019-05-14','2019-04-11','2018-12-20','2018-10-30','2018-09-26','2018-09-12','2018-09-04','2018-08-30','2018-08-01','2018-03-22','2018-02-16','2018-01-26','2017-12-20','2017-09-25','2017-06-30','2017-06-22','2017-05-26','2017-04-18','2017-03-29','2017-03-16']

## File under cleaning process
# Note: Can loop over dates, but special treatment in the next block suiting only up to 2023-11-21
date = dates[0]
url = f"https://www.ecfr.gov/api/versioner/v1/full/{date}/title-15.xml?appendix=Supplement+No.+4+to+Part+744&part=744"
response = requests.get(url)

In [322]:
# table
soup = bs(response.text,features='lxml')
find_table = soup.find('table')

# header
headers = find_table.find_all('th')
headers = [header.text.replace('\n',' ').strip() for header in headers]
df = pd.DataFrame(columns=headers)

# data
rows = find_table.find_all('tr')
filtered_df = pd.DataFrame() # working dataframe in debugging
list = [] # to check structural problem with the table
irregular = [] # to track unexpectedly empty cells
for row in rows:
    table_data = row.find_all('td')
    list.append(len(table_data))
    data = {header:j.text.strip() for j, header in zip(table_data, headers)}
    data = pd.DataFrame(data,index=[0])
    df = pd.concat([df, data],axis=0,ignore_index=True)
df = df.drop([0])
list = list[1:]

## special treatment of irregular rows
# check irregular table structure rows
for i in range(len(list)):
    if not list[i] == 5:
        irregular.append(i)
for index, row in df.iterrows():
    if not len(df.loc[index,'Federal Register citation']):
        irregular.append(index)
irregular.sort()
# show warning
print(f"Warning! Irregular of table happens in row(s) {irregular}")

# treatment of China Aerospace Science and Industry Corporation Second Academy and its subordinates, row 282-285
ind = df[df['Entity'].str.contains('China Aerospace Science and Industry Corporation Second Academy')].index[0]
df['Entity'][ind] = df['Entity'][ind] + '\n' + df['Entity'][ind+1] + '\n' + df['Entity'][ind+2] + '\n' + df['Entity'][ind+3]
df = df.drop([ind + i for i in range(1,4)]).reset_index(drop=True)

# treatment of Huawei and its affiliated entities, row 496
ind = df[df['Entity'].str.contains('Affiliated entities')].index[0]
df['Entity'][ind-1] = df['Entity'][ind-1] + '\n' + df['Entity'][ind]
df = df.drop([ind]).reset_index(drop=True)

# wrong new line of ICSOSO, row 502
ind = df[df['Entity'].str.contains('ICSOSO')].index[0]
df['Entity'][ind] = df['Entity'][ind] + df['Entity'][ind+1]
df['License requirement'][ind] = df['License requirement'][ind+1]
df['License review policy'][ind] = df['License review policy'][ind+1]
df['Federal Register citation'][ind] = df['Federal Register citation'][ind+1]
df = df.drop([ind+1]).reset_index(drop=True)


# fill empty fed registration rate, row 507
df.loc[df['Entity'].str.contains('ICSOSO'),'Federal Register citation'] = '87 FR 38925, 6/30/22.' # checked manually on https://www.federalregister.gov/documents/2023/09/27/2023-21080/addition-of-entities-and-revision-to-existing-entities-on-the-entity-list-removal-of-existing-entity

# treatment of irregular address, row 1674
ind = df.loc[df['Entity'].str.contains('Institute of Physics Named After P.N. Lebedev of the Russian Academy of Sciences')].index[0]
df['Entity'][ind] = df['Entity'][ind] + '\n\n' + df['Entity'][ind+1]
df = df.drop([ind+1]).reset_index(drop=True)

## treatment of Aerofalcon, row 2389
## This part solved later by the database
# mislocation_text = 'Aerofalcon'
# ind = df[df['License requirement'].str.contains(mislocation_text)].index
# grandchild = soup.find_all(lambda tag: len(tag.find_all()) == 0 and mislocation_text in tag.text)[0]
# country = grandchild.find_previous_siblings()[1].text
# table_data = grandchild.find_next_siblings()
# data = {
#     headers[0]:country,
#     headers[1]:grandchild.text,
#     headers[2]:table_data[0].text,
#     headers[3]:table_data[1].text,
#     headers[4]:table_data[2].text
# }
# for key, value in data.items():
#     df.loc[ind,key] = value.strip()

## Report total number of entities:
print(f"There are {len(df)} entities in total.")

## Fill down country
df['Country'] = df['Country'].replace('',np.nan).ffill(axis=0)

## Sanitize encoding problem
# the xml is in utf-8 but jupyter notebook processes and decodes it and saves it into csv in an ASCII way
# problems in License requirement
df['License requirement'] = df['License requirement'].str.replace('\xc2\xa7','section') # section symbol §

# problems in License review policy
df['License review policy'] = (df['License review policy']
                               .str.replace('\xc2\xa7\xc2\xa7','\xc2\xa7') # section symbol §
                               .str.replace('\xc2\xa7','section')) # section symbol §

# problems in Entity
clean_entity = {
    '\xE2\x80\x94':'-', # em dash into minus
    '\xE2\x80\x93':'-', # en dash into minus
    '\xE2\x80\x90':'-', # hyphen into minus
    '\xE2\x80\x9C':'\"', # double turned comma quotation into quotation
    '\xE2\x80\x9D':'\"', # double comma quotation into quotation
    '\xc2\xba':'', # MASCULINE ORDINAL INDICATOR
    '\xc3\x81':'A', # Á into A
    '\xc3\x89':'E', # É into E
    '\xc3\xa1':'a', # á into a
    '\xc3\xa3':'a', # ã into a
    '\xc3\xa4':'ae', # ä into ae
    '\xc3\xa9':'e', # é into e
    '\xc3\xb3':'o', # ó into o
    '\xc3\xb5':'o', # õ into o
    '\xc3\xb6':'o', # ö into o
    '\xc3\xbc':'ue', # ü into ue
    '\xc5\x9e':'S', # Ş into S
    '\xc5\x9f':'s', # ş into s
}
for key, item in clean_entity.items():
    df['Entity'] = df['Entity'].str.replace(key,item)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Warning! Irregular of table happens in row(s) [283, 284, 285, 496, 507, 1674]
There are 2712 entities in total.


In [323]:
# ## Get the name, alias, address and subordinate pattern with OpenAi
# import openai
# client = openai.OpenAI()

# completion = client.chat.completions.create(
#     model = 'gpt-3.5-turbo-1106',
#     messages = [
#         # {
#         #     'role':'system',
#         #     'content':'You will be given a series of strings that is a column of a dataframe. Your job is to read the strings, classify entity name, alias, subordinate/ affiliate and address, and conclude patterns of each classification in regular expression. You can try to classify the four elements by whether there exists either of them in the strings.'
#         # },
#         # {
#         #     'role':'user',
#         #     'content':f"The series to read is {df['Entity']}. Since the subordinates may also have aliases, and that there may be entities with neither subordinates nor aliases, you can first try to separate out name, and extract subordinates from the rest of the str ing, and extract alias of the entity but leave the alias of subordinates together into the list of subordinates, and finally get the address."
#         # }
#         {
#             'role':'user',
#             'content':f"read string {df['Entity'][303]},{df['Entity'][431]} and {df['Entity'][434]}. jupyter notebook displays what should have been dash, hyphen or minus as 'â\x80\x94' or 'â' where the space is not space but something else unknown. Tell me what it is"
#         }
#     ],
#     temperature=0,
#     # max_tokens=256,
#     top_p=1,
#     frequency_penalty=0,
#     presence_penalty=0
# )

# pd.options.display.max_colwidth = 1000000
# # with open('gpt_create_pattern.py', 'w') as file:
# #     file.write(completion.choices[0].message.content)
# print(completion.choices[0].message.content)
# print(completion.usage)

In [324]:
## Split Entity for name, address, alias, and subordinates
# Initialize Name, Address, Alias and Subordinate
df['Name'] = df['Alias'] = df['Address'] = df['Subordinate'] = 'NaN'

# identify two entities which are subordinates of CETC-7
destin_index = df[df['Entity'].str.contains('\(CETC-7\)')].index.item()
sub_indices = df[df['Entity'].str.contains('CETC-7',regex=False)].index.to_list()[1:]
df['Subordinate'][destin_index] = []
for i in range(len(sub_indices)):
    index = sub_indices[i]
    # sub = re.split(r"\(a subordinate institute",df['Entity'][index])[0]
    sub = df['Entity'][index]
    df['Subordinate'][destin_index].append(sub)
    df = df.drop(index)

# identify subordinates of CETC 13
indices = df[df['Entity'].str.contains('\(CETC 13\)')].index.to_list()
destin_index = indices[0]
sub_indices = indices[1:]
df['Subordinate'][destin_index] = []
for i in range(len(sub_indices)):
    index = sub_indices[i]
    sub = re.split(r"subordinate institution:\n?",df['Entity'][index])[1]
    # sub = df['Entity'][index]
    df['Subordinate'][destin_index].append(sub)
    df = df.drop(index)

# printing total number of independent entities in total
print(f"There are {len(df)} independent entities in total.")

There are 2698 independent entities in total.


In [325]:
## special treatment of irregular entities w.r.t. alias

# 1) for those with parentheses around the alias
# Mesbah Energy Company
ind = df.loc[df['Entity'].str.contains('Mesbah Energy Company')].index[0]
df['Entity'][ind] = re.sub(' \(a.k.a. "MEC"\)','(MEC)',df['Entity'][ind])

# Parto System Tehran
ind = df.loc[df['Entity'].str.contains('Parto System Tehran')].index[0]
df['Entity'][ind] = re.sub(r'\s\(a.k.a., Rayan Parto System Tehran and Rayane Parto System Tehran\),',' a.k.a., the following two aliases:\n-Rayan Parto System Tehran; and\n-Rayane Parto System Tehran\n\n',df['Entity'][ind])

# Paya Electronics Complex
ind = df.loc[df['Entity'].str.contains('Paya Electronics Complex')].index[0]
df['Entity'][ind] = re.sub(r'\s\(a.k.a., Paya Complex\), ',' a.k.a., the following one alias:\n-Paya Complex\n\n',df['Entity'][ind])

# Sabanican Company
ind = df.loc[df['Entity'].str.contains('Sabanican Company')].index[0]
df['Entity'][ind] = re.sub(r'\s\(a.k.a., Sabanican Pad Co.\),',', a.k.a., the following one alias:\n-Sabanican Pad Co.\n\n',df['Entity'][ind])

# Iman Group
ind = df.loc[df['Entity'].str.contains('Iman Group')].index[0]
df['Entity'][ind] = re.sub(', a.k.a., the following one\nalias:',', a.k.a., the following one alias:',df['Entity'][ind])

# Almaz-Antey Air Defense Concern Main System Design Bureau
ind = df.loc[df['Entity'].str.contains('Almaz-Antey Air Defense Concern Main System Design Bureau')].index[0]
df['Entity'][ind] = re.sub(r' a.k.a. GSKB\)',' a.k.a. GSKB',df['Entity'][ind])
df['Entity'][ind] = re.sub(' a.k.a.','\n-',df['Entity'][ind])
df['Entity'][ind] = re.sub(' f.k.a., ','\n-',df['Entity'][ind])
df['Entity'][ind] = re.sub(r'\s\(a.k.a.,',', a.k.a., the following two aliases:',df['Entity'][ind])

# Kalinin Machine Plant
ind = df.loc[df['Entity'].str.contains('Kalinin Machine Plant')].index[0]
df['Entity'][ind] = re.sub(r' Ekaterinburg\)',' Ekaterinburg',df['Entity'][ind])
df['Entity'][ind] = re.sub(' a.k.a.','\n-',df['Entity'][ind])
df['Entity'][ind] = re.sub(r'\s\(a.k.a.,',', a.k.a., the following two aliases:',df['Entity'][ind])

# Mytishchinski Mashinostroitelny Zavod, OAO
ind = df.loc[df['Entity'].str.contains('Mytishchinski Mashinostroitelny Zavod, OAO')].index[0]
df['Entity'][ind] = re.sub(r" Mashinostroitelny ZAVOD'\)"," Mashinostroitelny ZAVOD'",df['Entity'][ind])
df['Entity'][ind] = re.sub(' a.k.a.','\n-',df['Entity'][ind])
df['Entity'][ind] = re.sub(r'\s\(a.k.a.,',', a.k.a., the following two aliases:',df['Entity'][ind])

# Tikhomirov Scientific Research Institute of Instrument Design
ind = df.loc[df['Entity'].str.contains('Tikhomirov Scientific Research Institute of Instrument Design')].index[0]
df['Entity'][ind] = re.sub(r" Design.\)"," Design.'",df['Entity'][ind])
df['Entity'][ind] = re.sub(' a.k.a.','\n-',df['Entity'][ind])
df['Entity'][ind] = re.sub(' f.k.a., ','\n-',df['Entity'][ind])
df['Entity'][ind] = re.sub(r'\s\(a.k.a.,',', a.k.a., the following two aliases:',df['Entity'][ind])

# Wong Yuh Lan
ind = df.loc[df['Entity'].str.contains('Wong Yuh Lan')].index[0]
df['Entity'][ind] = re.sub(r'\s\(a.k.a.,',', a.k.a., the following two aliases:',df['Entity'][ind])
df['Entity'][ind] = re.sub(r" Huang Yulan, Jancy Wong and Yuh Lan Wong\),","\n-Huang Yulan;\n-Jancy Wong; and\n-Yuh Lan Wong\n\n",df['Entity'][ind])

# Aletra General Trading
ind = df.loc[df['Entity'].str.contains('Aletra General Trading')].index[0]
df['Entity'][ind] = re.sub(r'\s\(a.k.a.,',', a.k.a., the following two aliases:',df['Entity'][ind])
df['Entity'][ind] = re.sub(r"Erman & Sultan Trading Co.\), ","\n-Erman & Sultan Trading Co.\n\n",df['Entity'][ind])

# 2) rows with a.k.a./ f.k.a. as delimiter of alias
aka_entities = ['Dolgoprudny Research Production Enterprise','Gazprom Neft','Gazprom, OAO','Lukoil, OAO','Rosneft\s\(','Open Joint Stock Company Surgutneftegas']
for entity in aka_entities:
    ind = df.loc[df['Entity'].str.contains(entity)].index[0]
    df['Entity'][ind] = re.sub(r'f.k.a.','\n-',df['Entity'][ind])
    df['Entity'][ind] = re.sub(r'a.k.a.','\n-',df['Entity'][ind])
    df['Entity'][ind] = re.sub(r'\s\(',', a.k.a., the following several aliases:',df['Entity'][ind])
    df['Entity'][ind] = re.sub(r'\)','',df['Entity'][ind])

# 3.1) rows with comma as delimiter of alias, with street in address
comma_entities_1 = ['Encyclopedia Electronics Center','Industrial Establishment of Defense']
# Encyclopedia Electronics Center
ind = df.loc[df['Entity'].str.contains('Encyclopedia Electronics Center')].index[0]
result = re.split(', Mosalam Baroudi Street,', df['Entity'][ind],)
temp_entity, temp_address = result
temp_address = '\n\nMosalam Baroudi Street,' + temp_address
result = re.split(', a.k.a.,', temp_entity)
name, temp_alias = result
name = name + ', a.k.a., the following several aliases:\n-'
temp_alias = re.sub('\,',';\n-',temp_alias)
df['Entity'][ind] = name + temp_alias + temp_address

# Industrial Establishment of Defense
ind = df.loc[df['Entity'].str.contains('Industrial Establishment of Defense')].index[0]
result = re.split(', Al Thawraa Street,', df['Entity'][ind],)
temp_entity, temp_address = result
temp_address = '\n\nAl Thawraa Street,' + temp_address
result = re.split(', a.k.a.,', temp_entity)
name, temp_alias = result
name = name + ', a.k.a., the following several aliases:\n-'
temp_alias = re.sub('\,',';\n-',temp_alias)
df['Entity'][ind] = name + temp_alias + temp_address

# 3.2) rows with comma as delimiter of alias, without street in address
comma_entities_2 = ['Higher Institute of Applied Science and Technology','Scientific Studies and Research Center \(SSRC\),','National Standards and Calibration Laboratory']
for entity in comma_entities_2:
    ind = df.loc[df['Entity'].str.contains(entity)].index[0]
    result = re.split(', P.O.', df['Entity'][ind])
    temp_entity, temp_address = result
    temp_address = '\n\nP.O.' + temp_address
    result = re.split(', a.k.a.,', temp_entity)
    name, temp_alias = result
    name = name + ', a.k.a., the following several aliases:\n-'
    temp_alias = re.sub('\,',';\n-',temp_alias)
    df['Entity'][ind] = name + temp_alias + temp_address

# 4) idiosyncratic cleaning
# Engineering Materials and Equipment Co.
ind = df.loc[df['Entity'].str.contains('Engineering Materials and Equipment Co\.')].index[0]
df['Entity'][ind] = re.sub('\n\n-EMEC,','\n-EMEC\n\n',df['Entity'][ind])

# Joseph Choi, aka Yo-so'p Ch'oe,
ind = df.loc[df['Entity'].str.contains("Joseph Choi, aka Yo-so'p Ch'oe")].index[0]
df['Entity'][ind] = re.sub(r"Joseph Choi, aka Yo-so\'p Ch\'oe,",'Joseph Choi, a.k.a., the following one alias:\n- Yosop Choe\n\n',df['Entity'][ind])

# Presto Freight International LLC.
ind = df.loc[df['Entity'].str.contains("Presto Freight International LLC")].index[0]
df['Entity'][ind] = re.sub(r"aka Presto Freight International LLC \(PFI\), ",'a.k.a., the following one alias:\n-Presto Freight International LLC (PFI)\n\n',df['Entity'][ind])

# Manufacturers Equipment Organization (MEO)
ind = df.loc[df['Entity'].str.contains("Manufacturers Equipment Organization")].index[0]
df['Entity'][ind] = re.sub(r"following one alias\: -MEO GMBH P\.O",'following one alias:\n-MEO GMBH\n\nP.O',df['Entity'][ind])

# JLD Technology
ind = df.loc[df['Entity'].str.contains("JLD Technology, Hong Kong")].index[0]
df['Entity'][ind] = re.sub(r"JLD Technology, Hong Kong Co\. Ltd",'JLD Technology Hong Kong Co. Ltd',df['Entity'][ind])

# Abbas Goldoozan
ind = df.loc[df['Entity'].str.contains("JLD Technology, Hong Kong")].index[0]
df['Entity'][ind] = re.sub(r"Abbas Goldoozan, Kimya",'Abbas Goldoozan Kimya',df['Entity'][ind])

# Thomas McGuinn
ind = df.loc[df['Entity'].str.contains("Thomas McGuinn")].index[0]
df['Entity'][ind] = re.sub(r"Thomas McGuinn a",'Thomas McGuinn, a',df['Entity'][ind])


In [326]:
## special treatment of irregular entities w.r.t. address
# Kral Aviation
ind = df.loc[df['Entity'].str.contains('Kral Aviation Ltd\. Senlikkoy Mah\,')].index[0]
df['Entity'][ind] = re.sub('Kral Aviation Ltd\. Senlikkoy Mah\,','Kral Aviation Ltd\.\, Senlikkoy Mah\,',df['Entity'][ind])

# Xinnlinx Electronics Pte Ltd
ind = df.loc[df['Entity'].str.contains('Xinnlinx Electronics Pte Ltd,')].index[0]
df['Entity'][ind] = re.sub('Xinnlinx Electronics Pte Ltd,','Xinnlinx Electronics Pte Ltd.,',df['Entity'][ind])

# Bel Huawei Technologies
ind = df.loc[df['Entity'].str.contains('Bel Huawei Technologies LLC')].index[0]
df['Entity'][ind] = re.sub('BellHuawei Technologies LLC. 5','BellHuawei Technologies LLC.\n\n 5',df['Entity'][ind])

# CJSC Zest
ind = df.loc[df['Entity'].str.contains('Zest Leasing')].index[0]
df['Entity'][ind] = re.sub('Zest Leasing\n','Zest Leasing\n\n',df['Entity'][ind])


In [327]:
# name only entities
name_only_entities = ['The following Department of Atomic Energy entities', 'Ali Mehdipour Omrani', 'Aref Bali Lashak', 'Kamran Daneshjou', 'Mehdi Teranchi', 'Sayyed Mohammad Mehdi Hadavi', r'Allied Trading Co$', 'Prime International', 'Unique Technical Promoters', 'Brian Douglas Woodford']
name_only_index = [df.loc[df['Entity'].str.contains(entity)].index[0] for entity in name_only_entities]
filtered_df = pd.DataFrame()

# drop comma before certain patterns
drop_comma = {
    r',\sI?i?nc\.':' Inc.',
    r',\sLTD\.': ' Ltd.',
    r',\sLtd\.': ' Ltd.',
    r',\sltd\.': ' Ltd.',
    r',\sLtd': ' Ltd.',
    r',\sLLC':' LLC.',
    r',\sLimited\.?': 'Limited',
    r',\sLtd\s\(HiSilicon\)':' Ltd (HiSilicon)',
    r', S\.A\.': ' S.A.',
    r',\sOOO': ' OOO'
}       

for index, row in df.iterrows():
    # clean entity
    # remove see 'alternate addresses' part
    df['Entity'][index] = re.sub(r"\(S?s?ee (A?a?lternate|also) addresse?s? \b(under|in)\b .+\)\.?","",df['Entity'][index])

    # jump for name only entities:
    if index in name_only_index:
        df['Name'][index] = df['Entity'][index]
        continue

    # drop comma before certain patterns
    for key, item in drop_comma.items():
        df['Entity'][index] = re.sub(key,item,df['Entity'][index])

    # standardize 'a.k.a., the following .* alias'
    # special treatment with more than one appearance
    df['Entity'][index] = re.sub(r' \(a.k.a., NEL Electronics Pte Ltd\),',' a.k.a., the following one alias:\n-NEL Electronics Pte Ltd\n\n',df['Entity'][index])
    df['Entity'][index] = re.sub(r'\s\(a.k.a.,? Lin Rongnan, Steven Lim and Yong Nam Lim\)',' a.k.a., the following three aliases:\n-Lin Rongnan;\n-Steven Lim; and\n-Yong Nam Lim\n\n',df['Entity'][index])

    # more general cases
    df['Entity'][index] = re.sub(r"including the following alias", "the following one alias",df['Entity'][index]) 
    df['Entity'][index] = re.sub(r"the following (\w+) entity", r"the following \1 alias",df['Entity'][index]) 
    df['Entity'][index] = re.sub(r"the following alias","the following one alias",df['Entity'][index])
    df['Entity'][index] = re.sub(r"and the following (\w+) aliases", "a.k.a., the following \1 alias",df['Entity'][index])
    df['Entity'][index] = re.sub(r"the follow three aliases", "the following three aliases",df['Entity'][index])
    df['Entity'][index] = re.sub(r"a\.k\.a\., following two aliases", "a.k.a., the following two aliases",df['Entity'][index])
    df['Entity'][index] = re.sub(r"a\.k\.a\., the one alias", "a.k.a., the following one alias",df['Entity'][index])
    df['Entity'][index] = re.sub(r", a\.k\.a\., as the following two aliases", ", a.k.a., the following two aliases",df['Entity'][index])
    df['Entity'][index] = re.sub(r"a.k.a.?,? the following (\w+) alias", r", a.k.a., the following \1 alias",df['Entity'][index])
    df['Entity'][index] = re.sub(r"a.k.a.?,? the following (\w+) alias,", r", a.k.a., the following \1 alias:",df['Entity'][index])
    df['Entity'][index] = re.sub(r"a.k.a. the following (\w+) alias", r", a.k.a., the following \1 alias",df['Entity'][index])
    df['Entity'][index] = re.sub(r"a.k.a., the following (\w+) aliases\n", r", a.k.a., the following \1 aliases:\n",df['Entity'][index])
    if 'a.k.a' in df['Entity'][index] and not 'the following' in df['Entity'][index]:
        df['Entity'][index] = re.sub(r"a\.k\.a\.?,?", "a.k.a., the following one alias:\n-",df['Entity'][index])
    if 'the following' in df['Entity'][index] and not 'a.k.a.' in df['Entity'][index]:
        df['Entity'][index] = re.sub(r"the following", ", a.k.a., the following",df['Entity'][index])
    
    # Idea: first get subordinates with 'subordinate institutions., then alias. Note that only in the occasion of 'subordinate institutions' are there extra characters after 'the following (\w) alias(es)?:', that is, the only case that does not obey the pattern that splits name and aliases.
    if 'subordinate institution' in df['Entity'][index].lower():
        name_pattern = r', a\.k\.a\., the following .* aliase?s?, .* subordinate institutions:'
        result = re.split(name_pattern,df['Entity'][index])
        if len(result) == 2:
            df['Name'][index], temp = result
            result = re.split(r"\bS?s?ubordinate i?I?nstitution\b:?", temp)
            if len(result) == 3:
                df['Subordinate'][index] = [result[1].strip()]
                temp_alias = result[0].strip()
                temp_address = result[2].strip()
            else:
                df['Subordinate'][index] = result[1:-1]
                temp_alias = result[0].strip()
                temp_address = result[-1].strip()
            # get alias
            if '\n-' in temp_alias:
                result = re.split(r'\n\-',temp_alias)
                if len(result) == 1:
                    df['Alias'][index] = result[0].strip()
                else:
                    if result[0] == '':
                        df['Alias'][index] = result[1:]
                    else:
                        df['Alias'][index] = result[0:]
            else: 
                df['Alias'][index] = temp_alias
            # get address
            if 'the following addresses apply to the entity' in temp_address.lower():
                address_pattern = 'The following addresses apply to the entity and .* subordinate institutions:\n?\n?'
                result = re.split(address_pattern,temp_address)
                if len(result) == 2:
                    temp_subordinate, df['Address'][index] = result
                    df['Subordinate'][index].append(temp_subordinate)
            else:
                result = re.split(r'\n\n',temp_address)
                temp_subordinate, df['Address'][index] = result
                df['Subordinate'][index].append(temp_subordinate)
        continue


    # possible delimiter for address: \n\n, comma, 'The following addresses apply to the entity and the two subordinate institutions:'; 
    # for name: 'a.k.a., the following', comma, 'subordinate institution'; 
    # for subordinates: 'subordinate institution', minus sign; 
    # for alias: minus sign.
    try:
        # split entity
        # with 'a.k.a., the following XXX alias(es):' get name
        name_pattern = r', a\.k\.a\., the following .* aliase?s?:'
        result = re.split(name_pattern, df['Entity'][index],1)
        if len(result) == 2:# case where there is alias
            df['Name'][index], temp = result
            # get subordinate
            if 'subordinate entit' in temp.lower():
                result = re.split(r'; and the following four subordinate entities:\n',temp)
                df['Alias'][index], temp_subordinate = result
                df['Alias'][index] = [df['Alias'][index]]
                # get address and subordinate
                result = re.split(r'\n\n',temp_subordinate)
                df['Subordinate'][index] = result[0].strip() + result[1].strip()
                # df['Subordinate'][index] = [df['Subordinate'][index]]
                df['Address'][index] = result[2].strip()
            elif 'affiliated entities:' in temp.lower():
                df['Entity'][index] = re.sub(', and to include the following addresses and the following 22 affiliated entities:','', df['Entity'][index])# huawei row 491
                # get subordinate
                temp, df['Subordinate'][index] = re.split(r'Affiliated entities:\n',temp)
                # get address
                temp_alias, df['Address'][index] = re.split(r'\n\n',temp)
                # get alias
                if '\n-' in temp_alias:
                    result = re.split(r'\n\-',temp_alias)
                    if len(result) == 1:
                        df['Alias'][index] = result[0].strip()
                    else:
                        if result[0] == '':
                            df['Alias'][index] = result[1:]
                        else:
                           df['Alias'][index] = result[0:]
                else: 
                    df['Alias'][index] = [temp_alias]
            else: # case where there is alias but no subordinates
                # get address
                result = re.split(r'\n\n',temp) #\n\n as delimiter for address
                if len(result) == 2:
                    temp_alias, df['Address'][index] = result
                else:
                    result = re.split(r'\,',temp,1) # comma as delimiter for address
                    if len(result) == 2:
                        temp_alias, df['Address'][index] = result
                    else:
                        result = re.split(r'\.',temp,1) # period as delimiter for address
                        if len(result) == 2:
                            temp_alias, df['Address'][index] = result
                        else:
                            filtered_df = pd.concat([filtered_df,row],axis=1).transpose() # to be fixed individually
                # get alias
                if '\n-' in temp_alias: # multiple aliases or '-' as delimiter
                    result = re.split(r'\n\-',temp_alias)
                    if len(result) == 1:
                        df['Alias'][index] = result[0].strip()
                    else:
                        if result[0] == '':
                            df['Alias'][index] = result[1:]
                        else:
                           df['Alias'][index] = result[0:]
                else: # one alias only with no '-' as delimiter
                    df['Alias'][index] = [temp_alias]
        else: # case where there is no alias
            name_pattern = r'\,'
            result = re.split(name_pattern,df['Entity'][index],1)
            if len(result) == 2:
                df['Name'][index], temp = result
                # get address with \n\n
                result = re.split(r'\n\n',temp)
                if len(result) == 2:
                    df['Address'][index] = result[1]
                else:
                    df['Address'][index] = temp.strip() # there is only entity name and address available
            else:
                name_pattern = r'\n\n'
                result = re.split(name_pattern,df['Entity'][index])
                if len(result) == 2:
                    df['Name'][index], df['Address'][index] = result
    except:
        print(f"An exception occurred on row {index}")

if not len(filtered_df) == 0:
    print(f"There are problems w.r.t. address and alias in entities in the following dataframe: {filtered_df}.")


In [328]:
## Clean name, alias, subordinate and address
for index, row in df.iterrows():
    try:
        # Cleaning Name
        df['Name'][index] = df['Name'][index].rstrip(', ')

        # Cleaning alias
        if not df['Alias'][index] == 'NaN':
            df['Alias'][index] = [x.replace('; and','') for x in df['Alias'][index]]
            df['Alias'][index] = [x.replace(';','') for x in df['Alias'][index]]
            # df['Alias'][index] = df['Alias'][index].apply(lambda x: x.replace(r'\s-',''))
            # df['Alias'][index] = df['Alias'][index].apply(lambda x: x.replace(r'^-',''))
            # df['Alias'][index] = df['Alias'][index].apply(lambda x: x.replace(r'^\s',''))  

        # Cleaning Address
        # df['Address'] = df['Address'].str.replace('','NaN')
        df['Address'][index] = df['Address'][index].strip()

        # # Cleaning Subordinate
        # df['Subordinate'][index] = df['Subordinate'][index].apply(lambda x: x.replace('\n',''))
        # df['Subordinate'][index] = df['Subordinate'][index].apply(lambda x: x.lstrip('-'))
    except:
        print(f"An exception occurred on row {index}")


In [329]:
## Split human and company with identifier as a new column
# keywords for firm, government and research institutes
firmwords = [# Typical firm suffix
            'llc','corp','system','industr','company','ltd','co.','group','factory','enterprise','association','jsc','plant','branch','limited','llp','associates','foundation','inc','sdn bhd','development','headquarter','gmbh','limited','private','oao','ooo','zao','s.a.',' ao',' ab',' oy',' sarl','s.a.l.','trust','fze','fzco','holding','sdn','contracting','complex',' ag','Aktsionernoe Obshchestvo','venture','ltda',
            # Geography
            'global','beijing','international',
            # Industry
                # construction and manufacture
                'construction', 'steel','engineer','metro','bridge','production','konstrukt','manufactur','fku uprdor','establishment','Al-Qertas','Vangurd Tec','ELPROM','Elara','Moselectronproekt',
                # energy
                'energy','dietsmannnile','nyakek and sons','Oranto Petroleum','sanco','surgutneftegas','rosneft','chernomorneftegaz',
                # pharma
                'pharm','ELEMED','medical','Xinjiang Silk Road BGI','rau',
                # IT
                'comput','semiconductor','electron','micro','radio','microwave','cloud','display','elec','infotec','video','tronic','chip','cyber','angstrem','interscan','Proven Glory','higon','sugon','IFLYTEK','netposa','sensenets','network','dji','kindroid','candiru','corad','MCST Lebedev','NPP Istok','Avanlane','Milur SA','elektronika','SMT-iLogic','streloy','Grant Instrument','elektro','proexcom',
                # tech in general
                'technolog','integra','tech','solution','tekno','armyfly',
                # trade
                'trad','service','export','import','logist',
                # transportation
                'aero','airline','aerospace','shipyard','ship','aircraft','flight','aviation','used car','motors','skylink','vehicle','bike','concord','UEC-Saturn','MPI VOLNA','FASTAIR','Aviazapchast PLC','PT Air',
                # chemistry
                'chemie','interlab','labinvest','femteco',
                # others
                'design','field','resort','focus middle east','cosmos','NM-Tekh','TROJANS','Pearl Coral 1173 CC','consulting',
             # Firm specific
             'huawei', 'nexus','proven honour','gazprom','oceanos','vad, ao','magnetar','apex','melkom','abris','dm link','sngb ao','jadeshine','sputnik','ikco','cytrox','serop','aviton','bitreit','mekom','mces','meo','satco','aquanika','stroygazmontazh','transoil','intelcom','zener','source','yaltinskaya kinodstudiya','otkrytoe aktsionernoe obshchestvo vneshneekonomicheskoe obedinenie tekhnopromeksport','zte','micado','ar kompozit kimya','Regionsnab','Adimir OU','UAB Pella-Fjord','Alfakomponent','The Mother Ark.','Hasa Nederland B.V.','AST Components','kvant',
             # AO prefix 
             'AO Kronshtadt','AO Rubin','AO Aviaagregat','AO PKK Milandr','AO Papilon','AO Geomir','AO SET-1'
             ]
institutewords = ['university','research center','institute','academy','laborator','nscc','advanced research','development center','TsKB MT Rubin','SDB IRE RAS']
govwords =['ministry','desto','paec','cnsim','state ','federal','bureau','intelligence','department','committee','defense','atomic',"people's republic",'glavgosekspertiza rossii','crimea','zorsecurity','police']
words_list = [firmwords,institutewords,govwords]
(list(map(str.lower, words)) for words in words_list)

# matching type
# Note: in this order to capture gov or institutes wrongly taken as firm
df['Type']='Human'
for index, row in df.iterrows():
    text = row['Name']
    if any (firmword in text.lower() for firmword in firmwords):
        df.at[index,'Type']='Firm'
    if any (govword in text.lower() for govword in govwords):
        df.at[index,'Type']='Government'
    if any (instituteword in text.lower() for instituteword in institutewords):
        df.at[index,'Type']='Research Institute'
        
# idiosyncratic tuning
firm_idio = ['The Jordanian Lebanese Company for Laboratory Instruments S.A.L.','Svyaz Design Bureau, OJSC']
df.loc[df['Name'].isin(firm_idio),'Type'] = 'Firm'

## Report number each type
types = ['Human','Firm','Government','Research Institute']
i = 0
type_count = types
for iter in types:
    type_count[i] = df['Type'].str.contains(iter).sum()
    print(f"There are {type_count[i]} entities of type {iter}.")
    i += 1

There are 484 entities of type Human.
There are 1842 entities of type Firm.
There are 125 entities of type Government.
There are 247 entities of type Research Institute.


In [330]:
## Join identifier
# Note: the identifier is from Consolidated Screening List from US government's International Trade Administration on https://www.trade.gov/consolidated-screening-list
# Note: the identifier is generally unique to name, but there are still entities with multiple addresses assigned different identifiers. I have manually checked and it appears to be the dataset's problem and they essentially refer to the same entities. Duplicate identifiers are dropped and only the first are kept.
idmatch = (pd
            .read_csv("consolidated2024-01-05.csv",usecols=['_id','name','source'])
            .astype(str)
            .drop_duplicates(subset="name",keep='first')
            .rename(columns={'name':'Name'})
            .set_index('Name'))
df = df.merge(idmatch, on='Name', how='left')

In [331]:
## Split Fed Register and Effective Date
# clean fed register:
fed_register_clean = {
      '88 FR 121582/27/23.':'88 FR 12158, 2/27/23.',
      ' FR FR ': ' FR '
}
df['temp'] = df['Federal Register citation'].replace(fed_register_clean,regex=True)

# get effective date
df['Federal Register citation'] = df['temp'].apply(lambda x: re.findall(r'(\d+ FR \d+)', x))
df['Effective Date'] = df['temp'].apply(lambda x: re.findall(r'(\d+/\d+/\d+)', x))

# to check if there is mismatch
for index, row in df.iterrows():
    if not len(df['Federal Register citation'][index]) == len(df['Effective Date'][index]):
         print(index)

# explode fed register into panel
temp_date = pd.DataFrame(df['Effective Date'])
temp_list = df[['Entity','Alias','Subordinate','Address']]
df = df.drop(['temp','Effective Date','Alias','Subordinate','Address'],axis=1)
columns = df.columns.values.tolist()
columns.remove('Federal Register citation')
df = (df
      .set_index(columns)
      .explode('Federal Register citation')
      .reset_index()
)
df['Effective Date'] = temp_date.explode('Effective Date').reset_index()['Effective Date']
df = pd.merge(df,temp_list, on='Entity', how='outer')

# clean date
for index, row in df.iterrows():
      try:
            temp = datetime.strptime(df['Effective Date'][index],'%m/%d/%y')
      except ValueError:
            temp = datetime.strptime(df['Effective Date'][index],'%m/%d/%Y')
      except TypeError:
           continue
      df['Effective Date'][index] = temp.strftime('%Y-%m-%d')
      

# Get year
df['Year'] = df['Effective Date'].str[:4]

In [332]:
# Re-order dataframe
df = (df
      .reindex(columns=['_id','Entity','Name','Type','Country','Federal Register citation','Effective Date','Year','License requirement','License review policy','Address','Alias','Subordinate'])
      # .loc[:,['Alias','Subordinate']].apply(lambda x: tuple(x))
      # .drop_duplicates()
      .reset_index(drop=True)
      )
df.index += 1

In [333]:
## Save to a new csv file
file = f"EntityList{date}.csv"
file_cleaned = file.replace('.csv','_cleaned.csv')
df.to_csv(f"{file_cleaned}")